Автор: Беляев Владислав Владиславович,

аспирант ФБИТ ИТМО второго года обучения,

табельный номер 244026

## Шаг 1: Обзор данных

In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv("./data.csv")

In [3]:
raw_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
raw_data = raw_data.drop(["education_id", "family_status_id"], axis=1)

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       21525 non-null  int64  
 1   days_employed  19351 non-null  float64
 2   dob_years      21525 non-null  int64  
 3   education      21525 non-null  object 
 4   family_status  21525 non-null  object 
 5   gender         21525 non-null  object 
 6   income_type    21525 non-null  object 
 7   debt           21525 non-null  int64  
 8   total_income   19351 non-null  float64
 9   purpose        21525 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 1.6+ MB


In [6]:
raw_data.describe()

,children,days_employed,dob_years,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,1.000000,2.265604e+06


Выводы по предварительному обзору данных:

1) признаки `days_employed` и `total_income` содержат пропущенные значения, которые необходимо будет заполнить в процессе предобработки

2) признак `days_employed` содержит аномальные значения, как отрицательные, так и чрезмерно большие - максимальное значение составляет ~401755 дней, что приблизительно равно 1100 годам; устранить аномалии необходимо перед заполнением пропущенных значений.

3) признак `dob_years` содержит значения меньше 18, что для задачи финансового скоринга слабо применимо.

## Шаг 2: Предобработка данных



### Исправление аномальных значений

In [7]:
raw_data = raw_data.loc[raw_data["dob_years"] >= 18]

In [8]:
raw_data["days_employed"] = raw_data["days_employed"].apply(
    lambda x: int(abs(x)) if not pd.isnull(x) else x
)

<ipython-input-8-1464610878>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["days_employed"] = raw_data["days_employed"].apply(


In [9]:
raw_data.loc[
    raw_data["days_employed"] / 365 > raw_data["dob_years"] - 14,
    "days_employed"
] = pd.NA

### Заполнение пропусков в численных признаках

In [10]:
from pandas.api.types import is_numeric_dtype

In [11]:
def get_unfilled_columns(dataframe: pd.DataFrame) -> list[str]:
    return [
        column for column in dataframe.columns
        if dataframe[column].isna().sum() != 0
    ]


def get_numerical_columns(dataframe: pd.DataFrame) -> list[str]:
    return [
        column for column in dataframe.columns
        if is_numeric_dtype(dataframe[column])
    ]

In [12]:
unfilled_columns = get_unfilled_columns(raw_data)
unfilled_numerical_columns = get_numerical_columns(raw_data[unfilled_columns])

unfilled_numerical_columns

['days_employed', 'total_income']

In [13]:
for column_name in unfilled_numerical_columns:
    raw_data[column_name] = (
        raw_data \
        .groupby("income_type")[column_name] \
        .transform(
            lambda x: x.fillna(x.median())
        )
    )

In [14]:
print(raw_data.groupby('income_type')['days_employed'].median())

income_type
безработный           NaN
в декрете          3296.0
госслужащий        2672.0
компаньон          1544.5
пенсионер             NaN
предприниматель     520.0
сотрудник          1573.0
студент             578.0
Name: days_employed, dtype: float64


In [15]:
"""
поскольку не во всех категориях дохода представляется возможным заполнить пропуски,
удалим пропущенные значения чтобы они не влияли на дальнейший анализ
"""
raw_data = raw_data.dropna()

### Исправление неявных дубликатов в категориальных признаках

In [16]:
from pandas.api.types import is_string_dtype

In [17]:
def analyze_categorical_columns(
    dataframe: pd.DataFrame,
    *,
    categorical_columns: list[str] | None = None,
) -> None:
    if categorical_columns is None:
        categorical_columns = filter(
            lambda column_name: is_string_dtype(dataframe[column_name]),
            dataframe.columns,
        )

    for column_name in categorical_columns:
        column_values = dataframe[column_name] \
            .value_counts(ascending=True) \
            .to_dict()

        print(column_name)
        for value_name, value_count in column_values.items():
            print(f"\t{value_name} : {value_count}")
        print()

In [18]:
analyze_categorical_columns(raw_data)

education
	УЧЕНАЯ СТЕПЕНЬ : 1
	ученая степень : 3
	НАЧАЛЬНОЕ : 10
	Начальное : 10
	НЕОКОНЧЕННОЕ ВЫСШЕЕ : 27
	Неоконченное высшее : 43
	начальное : 163
	ВЫСШЕЕ : 241
	Высшее : 244
	Среднее : 543
	СРЕДНЕЕ : 618
	неоконченное высшее : 631
	высшее : 4168
	среднее : 10884

family_status
	вдовец / вдова : 422
	в разводе : 968
	Не женат / не замужем : 2449
	гражданский брак : 3496
	женат / замужем : 10251

gender
	XNA : 1
	M : 6561
	F : 11024

income_type
	в декрете : 1
	студент : 1
	предприниматель : 2
	госслужащий : 1453
	компаньон : 5065
	сотрудник : 11064

purpose
	заняться образованием : 339
	получение высшего образования : 351
	получение дополнительного образования : 356
	профильное образование : 359
	образование : 363
	дополнительное образование : 365
	получение образования : 366
	приобретение автомобиля : 371
	сделка с автомобилем : 371
	высшее образование : 376
	на покупку автомобиля : 377
	автомобили : 382
	на покупку подержанного автомобиля : 389
	автомобиль : 391
	свой автомобиль 

In [19]:
raw_data["education"] = raw_data["education"].apply(str.lower)

<ipython-input-19-2142752574>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["education"] = raw_data["education"].apply(str.lower)


In [20]:
analyze_categorical_columns(raw_data, categorical_columns=["education"])

education
	ученая степень : 4
	начальное : 183
	неоконченное высшее : 701
	высшее : 4653
	среднее : 12045



In [21]:
def transform_purpose_value(purpose_value: str) -> str:
    if "свадьб" in purpose_value:
        return "свадьба"
    elif "образовани" in purpose_value:
        return "образование"
    elif "автомобил" in purpose_value:
        return "автомобиль"
    elif (
        "жиль" in purpose_value
        or "недвижимост" in purpose_value
    ):
        return "недвижимость"
    else:
        raise ValueError(f"Unexpected purpose: {purpose_value}")

In [22]:
raw_data["purpose"] = raw_data["purpose"].apply(transform_purpose_value)

<ipython-input-22-606263950>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["purpose"] = raw_data["purpose"].apply(transform_purpose_value)


In [23]:
analyze_categorical_columns(raw_data, categorical_columns=["purpose"])

purpose
	свадьба : 1902
	образование : 3283
	автомобиль : 3499
	недвижимость : 8902



### Удаление слабо представленных значений

In [24]:
analyze_categorical_columns(
    raw_data, categorical_columns=[
        *filter(
            lambda column_name: is_string_dtype(raw_data[column_name]),
            raw_data.columns,
        ),
        "children"
    ]
)

education
	ученая степень : 4
	начальное : 183
	неоконченное высшее : 701
	высшее : 4653
	среднее : 12045

family_status
	вдовец / вдова : 422
	в разводе : 968
	Не женат / не замужем : 2449
	гражданский брак : 3496
	женат / замужем : 10251

gender
	XNA : 1
	M : 6561
	F : 11024

income_type
	в декрете : 1
	студент : 1
	предприниматель : 2
	госслужащий : 1453
	компаньон : 5065
	сотрудник : 11064

purpose
	свадьба : 1902
	образование : 3283
	автомобиль : 3499
	недвижимость : 8902

children
	5 : 9
	-1 : 39
	4 : 40
	20 : 66
	3 : 322
	2 : 2022
	1 : 4526
	0 : 10562



In [25]:
raw_data = raw_data.loc[raw_data["gender"] != "XNA"]

In [26]:
analyze_categorical_columns(raw_data, categorical_columns=["gender"])

gender
	M : 6561
	F : 11024



In [27]:
raw_data = raw_data.loc[~raw_data["income_type"].isin(["студент", "в декрете", "предприниматель"])]

In [28]:
analyze_categorical_columns(raw_data, categorical_columns=["income_type"])

income_type
	госслужащий : 1453
	компаньон : 5064
	сотрудник : 11064



In [29]:
raw_data = raw_data.loc[~raw_data["children"].isin([-1, 20])]

In [30]:
analyze_categorical_columns(raw_data, categorical_columns=["children"])

children
	5 : 9
	4 : 40
	3 : 322
	2 : 2021
	1 : 4526
	0 : 10558



In [31]:
"""
т.к. значение 'ученая степень' представлено слабо в наборе данных,
можно:
    1) удалить значения
    2) объединить с категорией "высшее"

выберем второй вариант
"""

raw_data["education"] = raw_data["education"].apply(
    lambda x: "высшее" if x == "ученая степень" else x
)

### Создание категориальных признаков

In [32]:
def transform_income_to_category(income_value: float) -> str:
    if 0 <= income_value <= 30_000:
        return "E"
    elif 30_000 < income_value <= 50_000:
        return "D"
    elif 50_000 < income_value <= 200_000:
        return "C"
    elif 200_000 < income_value <= 1_000_000:
        return "B"
    elif 1_000_000 < income_value:
        return "A"
    else:
        raise ValueError(f"Unexpected income value: {income_value}")

In [33]:
raw_data["income_category"] = raw_data["total_income"].apply(transform_income_to_category)

In [34]:
analyze_categorical_columns(raw_data, categorical_columns=["income_category"])

income_category
	E : 7
	A : 25
	D : 184
	B : 4437
	C : 12823



In [35]:
def transform_children_to_category(children_value: int) -> str:
    if children_value == 0:
        return "бездетный"
    elif children_value == 1 or children_value == 2:
        return "малодетный"
    elif children_value > 2:
        return "многодетный"
    else:
        raise ValueError(f"Unexpected children value: {children_value}")

In [36]:
raw_data["child_category"] = raw_data["children"].apply(transform_children_to_category)

In [37]:
analyze_categorical_columns(raw_data, categorical_columns=["child_category"])

child_category
	многодетный : 371
	малодетный : 6547
	бездетный : 10558



In [38]:
raw_data["age_category"] = raw_data["dob_years"].apply(lambda x: x // 10 * 10)

In [39]:
analyze_categorical_columns(raw_data, categorical_columns=["age_category"])

age_category
	10 : 14
	70 : 22
	60 : 531
	50 : 2969
	20 : 3145
	40 : 5194
	30 : 5601



### Удаление дубликатов

In [40]:
raw_data = raw_data.drop_duplicates(keep='first')

In [41]:
assert raw_data[raw_data.duplicated(keep=False)].shape[0] == 0

## Шаг 3: Исследование данных

In [42]:
data_to_analyze = pd.DataFrame()


data_to_analyze["age"] = raw_data["dob_years"]
data_to_analyze["age_category"] = raw_data["age_category"]
data_to_analyze["gender"] = raw_data["gender"]
data_to_analyze["education"] = raw_data["education"]
data_to_analyze["family"] = raw_data["family_status"]
data_to_analyze["child_category"] = raw_data["child_category"]
data_to_analyze["children"] = raw_data["children"]
data_to_analyze["income_type"] = raw_data["income_type"]
data_to_analyze["experience_years"] = raw_data["days_employed"].apply(
    lambda x: round(x / 365, 2)
)
data_to_analyze["income_category"] = raw_data["income_category"]
data_to_analyze["income_value"] = raw_data["total_income"].apply(
    lambda x: round(x, 2)
)
data_to_analyze["purpose"] = raw_data["purpose"]
data_to_analyze["debt"] = raw_data["debt"]


data_to_analyze.head(3)

,age,age_category,gender,education,family,child_category,children,income_type,experience_years,income_category,income_value,purpose,debt
0,42,40,F,высшее,женат / замужем,малодетный,1,сотрудник,23.12,B,253875.64,недвижимость,0
1,36,30,F,среднее,женат / замужем,малодетный,1,сотрудник,11.02,C,112080.01,автомобиль,0
2,33,30,M,среднее,женат / замужем,бездетный,0,сотрудник,15.41,C,145885.95,недвижимость,0


### Анализ должников по возрастным категориям

In [43]:
import matplotlib.pyplot as plt

In [44]:
debt_by_age_cat = data_to_analyze.groupby("age_category").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_age_cat["share_of_debtors"] = (
    debt_by_age_cat["total_debtors"] / debt_by_age_cat["total_borrowers"]
)

In [45]:
debt_by_age_cat

,age_category,total_borrowers,total_debtors,share_of_debtors
0,10,14,1,0.071429
1,20,3113,348,0.111789
2,30,5530,539,0.097468
3,40,5086,389,0.076484
4,50,2907,197,0.067767
5,60,524,30,0.057252
6,70,22,1,0.045455


В возрастных категориях от 30 лет количество должников как в абсолютном, так и в относительном выражении снижается, поэтому предпочтение стоит отдавать именно этой возрастной категории.

### Анализ должников по уровню образования

In [46]:
debt_by_education = data_to_analyze.groupby("education").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_education["share_of_debtors"] = (
    debt_by_education["total_debtors"] / debt_by_education["total_borrowers"]
)

In [47]:
debt_by_education

,education,total_borrowers,total_debtors,share_of_debtors
0,высшее,4590,249,0.054248
1,начальное,183,28,0.153005
2,неоконченное высшее,697,67,0.096126
3,среднее,11726,1161,0.099011


Как можно заметить, наименьший процент должников - в группе людей с высшим образованием, а наибольшее количество - среди имеющих только среднее образование.

Поэтому наличие высшего образования (как минимум, неоконченного) можно использовать для предварительной оценки кредитоспособности клиента

### Анализ должников по семейному положению

In [48]:
debt_by_family = data_to_analyze.groupby("family").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_family["share_of_debtors"] = (
    debt_by_family["total_debtors"] / debt_by_family["total_borrowers"]
)

In [49]:
debt_by_family

,family,total_borrowers,total_debtors,share_of_debtors
0,Не женат / не замужем,2414,256,0.106048
1,в разводе,961,71,0.073881
2,вдовец / вдова,418,26,0.062201
3,гражданский брак,3446,347,0.100696
4,женат / замужем,9957,805,0.080848


In [50]:
debt_by_children = data_to_analyze.groupby("children").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_children["share_of_debtors"] = (
    debt_by_children["total_debtors"] / debt_by_children["total_borrowers"]
)

In [51]:
debt_by_children

,children,total_borrowers,total_debtors,share_of_debtors
0,0,10361,859,0.082907
1,1,4461,426,0.095494
2,2,2005,190,0.094763
3,3,321,26,0.080997
4,4,39,4,0.102564
5,5,9,0,0.000000


In [52]:
debt_by_child_cat = data_to_analyze.groupby("child_category").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_child_cat["share_of_debtors"] = (
    debt_by_child_cat["total_debtors"] / debt_by_child_cat["total_borrowers"]
)

In [53]:
debt_by_child_cat

,child_category,total_borrowers,total_debtors,share_of_debtors
0,бездетный,10361,859,0.082907
1,малодетный,6466,616,0.095268
2,многодетный,369,30,0.081301


### Анализ должников по доходу

In [54]:
debt_by_income_type = data_to_analyze.groupby("income_type").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_income_type["share_of_debtors"] = (
    debt_by_income_type["total_debtors"] / debt_by_income_type["total_borrowers"]
)

In [55]:
debt_by_income_type

,income_type,total_borrowers,total_debtors,share_of_debtors
0,госслужащий,1438,86,0.059805
1,компаньон,4981,373,0.074885
2,сотрудник,10777,1046,0.097059


In [56]:
debt_by_income_category = data_to_analyze.groupby("income_category").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_income_category["share_of_debtors"] = (
    debt_by_income_category["total_debtors"] / debt_by_income_category["total_borrowers"]
)

In [57]:
debt_by_income_category

,income_category,total_borrowers,total_debtors,share_of_debtors
0,A,25,2,0.080000
1,B,4437,322,0.072572
2,C,12543,1167,0.093040
3,D,184,14,0.076087
4,E,7,0,0.000000


Как можно отметить, госслужащие являются группой со значительно более низким процентом должников. Однако по общему количеству пользователей данная группа значительно отстает от остальных.

Таким образом, следует сконцентрироваться на привлечении клиентов из числа госслужащих.  

### Анализ должников по цели займа

In [58]:
debt_by_purpose = data_to_analyze.groupby("purpose").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

debt_by_purpose["share_of_debtors"] = (
    debt_by_purpose["total_debtors"] / debt_by_purpose["total_borrowers"]
)

In [59]:
debt_by_purpose

,purpose,total_borrowers,total_debtors,share_of_debtors
0,автомобиль,3442,345,0.100232
1,недвижимость,8665,683,0.078823
2,образование,3220,321,0.099689
3,свадьба,1869,156,0.083467


Как можно заметить, недвижимость является преимущественной целью займов, при этом данная категория обладает наименьшем процентов должников.

По этой причине следует уделять развитие данному направлению.

In [60]:
data_to_analyze.loc[
    (data_to_analyze["age"] >= 30) & (data_to_analyze["education"] == "высшее")
].groupby("purpose").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

,purpose,total_borrowers,total_debtors
0,автомобиль,716,40
1,недвижимость,1823,85
2,образование,631,35
3,свадьба,416,20


In [61]:
data_to_analyze.loc[
    data_to_analyze["income_type"] == "госслужащий"
].groupby("purpose").agg(
    total_borrowers=("debt", "count"),
    total_debtors=("debt", "sum"),
).reset_index()

,purpose,total_borrowers,total_debtors
0,автомобиль,281,22
1,недвижимость,749,36
2,образование,252,21
3,свадьба,156,7


Кроме того, для рекомендуемых аудиторий с низким риском возникновения задолженности - 1) людей старше 30 с высшим образованием; и 2) госслужащих - спрос на данное направление превышает остальные вместе взятые. Что подтверждает необходимость развития займов на недвижимость.

## Заключение

Рекомендации по итогам анализа данных можно разделить на две группы: первая касается развитию кредитных продуктов исходя из анализа клиентов; вторая - необходимости модернизации процесса сбора данных.

Что касается клиентов кредитной организации, среди них, как и отмечалось ранее, можно выделить две группы лиц с относительно более низким шансом возникновения задолженности:

- лица старше 30 с высшим образованием;
- госслужащие.

Несмотря на то, что первая группа значительно представлена среди клиентов (около 20 процентов проанализированных клиентов), вторая же включает в себя только 8 процентов выборки.

Категория госслужащих наиболее заинтересована в займах на недвижимость. Аналогичный спрос отмечается и у первой категории клиентов. Поэтому развитие данного направления повысит количество клиентов из категорий с низким риском, что в целом позволит снизить риски задолженности.

Касательно клиентских данных, можно отметить два возможных направления его модернизации.

Первое - критичное - это проверка пользовательских данных. По итогам анализа данных, было выявлено большое количество неправильных значений. Наиболее проблемным пунктом оказались данные о стаже занятости. Данный пункт содержал большое количество пропусков и аномальных (отрицательных или чрезвычайно высоких) значений. При попытке восстановления данных значений было выявлено, что преимущественное количество аномалий связано с клиентами пенсионного возраста, из-за чего стоит тщательно проверять информацию среди подобных клиентов. Кроме того, аномальные значения отмечались среди количества детей у клиентов (например, -1 и 20) а также возраста клиентов (0-10 лет).

Второе - оптимизация заполнения полей анкеты клиентов.

Поле "образование" содержит ограниченное количество вариантов - "начальное", "среднее", "неоконченное высшее" и "высшее". При этом клиенты заполняют данные различным способом - верхним/нижним регистром или с заглавной буквы, что приводит к необходимости постобработки данного поля.

Поле "цель" пишется клиентами в свободной форме, хотя при анализе можно выделить 4 крупных группы - организация свадьбы, оплата образования, операции с недвижимостью и автомобилями.

Замена способа заполнения данных полей анкеты с ввода данных клиентами в свободной форме на выбор из предварительно предложенных значений избавит от необходимости в постобработке данных в процессе анализа.